# Load the data

In [ ]:
# Mount Google Drive locally 
# Using the instructions found here https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA&line=1&uniqifier=1

Mounted at /content/drive


In [ ]:
# Load the pickled dataframe using the "read_pickle()"" function from "pandas"
# df = ...

In [ ]:
# install AIF360
!pip install aif360

# Tune the thershold parameter

In [ ]:
# Instantiate the classifier
from sklearn.linear_model import LogisticRegression
# lr = ...

# instantiate the cross-validation scheme
from sklearn.model_selection import StratifiedKFold
# mv = ...

# setup the performance metrics to be computed
from sklearn import metrics
# perf_metrics = {"Accuracy": ..., 
#                 "Precision": ..., 
#                 "Recall": ...,
#                 "AUC": ..., 
#                 "F1-Score": ..., 
#                 }

# Set up the fairness metrics to be computed using AIF360
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

# privileged_group = ...
# unprivileged_group = ...

In [ ]:
# Tune the threshold using 10 fold cross validation

thresholds = [0.3, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7]
import pandas as pd
import numpy as np
k, i = True, 0

# instantiating X
# X = ...

# instantiating the target variable
# y = ...

for fold_threshold in thresholds:
  av_f1 = 0
  av_acc = 0
  av_auc = 0
  av_precision = 0
  av_recall = 0
  av_statpar = 0
  av_av_odds = 0


  for (train, test) in mv.split(X, y):

      # fit model
      # lr = lr.fit(...)
      
      # get predictions in the test set
      # ypred_prob = lr.predict_proba ... # get probabilities
      # ypred_class = lr.predict ...

      # compute performance metrics
      metrics = []
      metrics += [['Threshold', fold_threshold]]
      dataset = X.iloc[test].copy()
      dataset[df.columns[-1]] = np.expand_dims(y.iloc[test], axis=1)
      # dataset = BinaryLabelDataset(...)
      dataset_pred = X.iloc[test].copy()
      dataset_pred[df.columns[-1]] = np.expand_dims(ypred_class, axis=1)
      # dataset_pred = BinaryLabelDataset(...)
      # metric_CM = ClassificationMetric(...)

      av_f1 += perf_metrics["F1-Score"](y.iloc[test].values.ravel(), ypred_class)
      av_acc += perf_metrics["Accuracy"](y.iloc[test].values.ravel(), ypred_class)
      av_auc += perf_metrics["AUC"](y.iloc[test].values.ravel(), ypred_class)
      av_precision += perf_metrics["Precision"](y.iloc[test].values.ravel(), ypred_class)
      av_recall += perf_metrics["Recall"](y.iloc[test].values.ravel(), ypred_class)
      av_statpar = metric_CM.statistical_parity_difference()
      av_av_odds = metric_CM.average_odds_difference()

    
  # Compute fairness metrics
  # metrics += [['Threshold',...]]
  # metrics += [['F1',...]]
  # metrics += [['Accuracy',...]]
  # metrics += [['AUC',...]]
  # metrics += [['Precision',...]]
  # metrics += [['Recall',...]]
  # metrics += [['Statistical Parity Difference',...]]
  # metrics += [['Average Odds Difference',...]]


  # concatenate results
  df_m = pd.DataFrame(metrics, columns=["Metric", "Value"])
  df_m["Fold"] = i
  i += 1
  if k:
      df_metrics = df_m.copy()
      k=0
  else:
      df_metrics = pd.concat([df_metrics, df_m.copy()], axis=0, ignore_index=True)

In [ ]:
# Display metrics
# df_pivot = ...
df_pivot

In [ ]:
# Plot metrics against the threshold values
import seaborn as sns
y_vars = ['Threshold']
x_vars = ['Average Odds Difference', 'Statistical Parity Difference', 'AUC', 'Accuracy', 'F1', 'Precision', 'Recall']

g = sns.PairGrid(df_pivot, x_vars=x_vars, y_vars=y_vars)
g.map(sns.scatterplot)

# Re-run post-processing with two different classification thresholds.

In [ ]:
# Instantiate the classifier
from sklearn.linear_model import LogisticRegression
# lr = ...

# instantiate the cross-validation scheme
from sklearn.model_selection import StratifiedKFold
# mv = ...

# setup the performance metrics to be computed
from sklearn import metrics
# perf_metrics = {"Accuracy": ..., 
#                 "Precision": ..., 
#                 "Recall": ...,
#                 "AUC": ..., 
#                 "F1-Score": ..., 
#                 }

# Set up the fairness metrics to be computed using AIF360
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

# privileged_group = ...
# unprivileged_group = ...

In [ ]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing

# Learn parameters to equalize odds and apply to create a new dataset
# epp = EqOddsPostprocessing(...)


In [ ]:
# Train a logistic regression classifier with two different thresholds

thresholds = [0.45, 0.5]
import pandas as pd
import numpy as np
k, i = True, 0

# instantiating X
# X = ...

# instantiating the target variable
# y = ...

for fold_threshold in thresholds:
  av_f1 = 0
  av_acc = 0
  av_auc = 0
  av_precision = 0
  av_recall = 0
  av_statpar = 0
  av_av_odds = 0

  for (train, test) in mv.split(X, y):

      # fit model
      # lr = lr.fit(...)
      
      # get predictions in the test set
      # ypred_prob = lr.predict_proba ...
      # ypred_class = ...
      # y_test = ...
      # X_test = ...

      # fit post-processing using results from 60% of the test set
      test_pct = 0.4
      n = int(len(y_test))
      n_2 = int(n* (1-test_pct))
      indices = np.random.permutation(n)
      pp_indices = indices[:n_2]
      test_indices = indices[n_2:]

      pp_dataset = X_test.iloc[pp_indices].copy()
      pp_dataset[df.columns[-1]] = np.expand_dims(y_test.iloc[pp_indices], axis=1)
      pp_dataset_BLD = BinaryLabelDataset(df=pp_dataset, label_names=['action_taken_name'], protected_attribute_names=['applicant_sex_name_Female'])

      pp_pred = X_test.iloc[pp_indices].copy()
      pp_pred[df.columns[-1]] = np.expand_dims(ypred_class[pp_indices], axis=1)
      pp_pred_BLD = BinaryLabelDataset(df=pp_pred, label_names=[df.columns[-1]], protected_attribute_names=['applicant_sex_name_Female'])

      epp = epp.fit(pp_dataset_BLD, pp_pred_BLD)

      # Use epp to post-process predictions on the other 40% of the test set
      actuals_test = X_test.iloc[test_indices].copy()
      actuals_test[df.columns[-1]] = np.expand_dims(y_test.iloc[test_indices], axis=1)
      actuals_test_BLD = BinaryLabelDataset(df=actuals_test, label_names=['action_taken_name'], protected_attribute_names=['applicant_sex_name_Female'])
      pred_test = X_test.iloc[test_indices].copy()
      pred_test[df.columns[-1]] = np.expand_dims(ypred_class[test_indices], axis=1)
      pred_test_BLD = BinaryLabelDataset(df=pred_test, label_names=[df.columns[-1]], protected_attribute_names=['applicant_sex_name_Female'])

      transf_pred_test_BLD = epp.predict(pred_test_BLD)

      # compute performance metrics
      metrics = []
      metrics += [['Threshold', fold_threshold]]
      # metric_CM = ClassificationMetric(...)

      av_f1 += perf_metrics["F1-Score"](y.iloc[test].values.ravel(), ypred_class)
      av_acc += perf_metrics["Accuracy"](y.iloc[test].values.ravel(), ypred_class)
      av_auc += perf_metrics["AUC"](y.iloc[test].values.ravel(), ypred_class)
      av_precision += perf_metrics["Precision"](y.iloc[test].values.ravel(), ypred_class)
      av_recall += perf_metrics["Recall"](y.iloc[test].values.ravel(), ypred_class)
      av_statpar = metric_CM.statistical_parity_difference()
      av_av_odds = metric_CM.average_odds_difference()

    
  # Compute fairness metrics
  # metrics += [['Threshold', ...]]
  # metrics += [['F1', ...]]
  # metrics += [['Accuracy', ...]]
  # metrics += [['AUC', ...]]
  # metrics += [['Precision', ...]]
  # metrics += [['Recall', ...]]
  # metrics += [['Statistical Parity Difference', ...]]
  # metrics += [['Average Odds Difference', ...]]


  # concatenate results
  df_m = pd.DataFrame(metrics, columns=["Metric", "Value"])
  df_m["Fold"] = i
  i += 1
  if k:
      df_metrics = df_m.copy()
      k=0
  else:
      df_metrics = pd.concat([df_metrics, df_m.copy()], axis=0, ignore_index=True)

In [ ]:
# Display metrics
#  df_pivot = ...
df_pivot

The model that minimises false-negatives is the model with a threshold of 0.45.